In [1]:
import numpy as np
import pandas as pd
import pickle
import random
from glob import glob
from utils import *

data_folder = "/home/cbel/Desktop/zhe/data"

In [2]:
def load_data(df, imgs_path, is_malignancy):
    
    if is_malignancy == 1:
        index = df[df["malignancy"]==1]["nodule_number"].values
    if is_malignancy == 0:
        index = df[df["malignancy"]==0]["nodule_number"].values
    if is_malignancy == 2:
        index = df[df["malignancy"]==2]["nodule_number"].values
    
    all_id = []
    for i in range(len(imgs_path)):
        idx = int(imgs_path[i][-8:-4])
        if idx in index:
            all_id.append(idx)
    return all_id

In [3]:
def split_10(index):
    split = 10
    total_num = len(index)
    partition_num = total_num // 10
    remainder = total_num % 10
    tenFold = {}
    tmp_index = index.copy()
    num = 0
    for fold in range(10):
        subFold = []
        for i in range(partition_num):
            num += 1
            subFold.append(tmp_index.pop(0))
        if remainder > 0:
            subFold.append(tmp_index.pop(0))
            remainder -= 1
        tenFold[fold] = subFold
    return tenFold

In [4]:
df = pd.read_csv(data_folder + "/final_annotation.csv")
imgs_path = glob(data_folder + "/multi_view/original/*.npy")
imgs_path.sort()
random.seed(11)
random.shuffle(imgs_path)
be_index = load_data(df, imgs_path, 0)
ma_index = load_data(df, imgs_path, 1)
in_index = load_data(df, imgs_path, 2)

In [5]:
be_tenFold = split_10(be_index)
ma_tenFold = split_10(ma_index)

In [6]:
tenFold = {}
for i in range(10):
    fold = [0 , 1, 2, 3, 4, 5, 6, 7, 8, 9]
    tenFold[i] = {}
    for split in ["val", "train"]:
        if split == "val":
            tenFold[i][split] = be_tenFold[i] + ma_tenFold[i]
        else:
            fold.remove(i)
            tenFold[i][split] = []
            tmp = [be_tenFold[j] + ma_tenFold[j] for j in fold]
            for sub_tmp in tmp:
                for k in sub_tmp:
                    tenFold[i][split].append(k) 
                
tenFold_pickle = open(data_folder + "/tenFold.pickle", "wb")
pickle.dump(tenFold, tenFold_pickle)
tenFold_pickle.close()